<a href="https://colab.research.google.com/github/bushragit/Medical_cases-ner-/blob/main/ner(c).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#download required libraries
!pip install langchain-huggingface
!pip install huggingface_hub
!pip install transformers
!pip install bits and bytes
!pip install langchain
!pip install accelerate

In [ ]:
from transformers import pipeline

In [ ]:
pipe= pipeline(model="Clinical-AI-Apollo/Medical-NER")

config.json:   0%|          | 0.00/5.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/736M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

In [ ]:
from langchain import  LLMChain, PromptTemplate
from transformers import AutoTokenizer, AutoModelForTokenClassification

In [ ]:
model_name = "Clinical-AI-Apollo/Medical-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

In [ ]:
# Define the NER pipeline
pipe = pipeline("token-classification", model=model, tokenizer=tokenizer)

In [ ]:
description=input("enter case history? ")

enter case history? A 22-year-old male presented to his physician complaining of a fever and sore throat with enlarged, tender lymph nodes in his neck for the last 24 hours. He was now feeling very ill, and his symptoms were getting worse.


In [ ]:
ner_results= pipe(description)

In [ ]:
ner_results

[{'entity': 'B-AGE',
  'score': 0.7957225,
  'index': 2,
  'word': '▁22',
  'start': 1,
  'end': 4},
 {'entity': 'I-AGE',
  'score': 0.88713187,
  'index': 3,
  'word': '-',
  'start': 4,
  'end': 5},
 {'entity': 'I-AGE',
  'score': 0.9446201,
  'index': 4,
  'word': 'year',
  'start': 5,
  'end': 9},
 {'entity': 'I-AGE',
  'score': 0.8576373,
  'index': 5,
  'word': '-',
  'start': 9,
  'end': 10},
 {'entity': 'I-AGE',
  'score': 0.9251152,
  'index': 6,
  'word': 'old',
  'start': 10,
  'end': 13},
 {'entity': 'B-SEX',
  'score': 0.51480496,
  'index': 7,
  'word': '▁male',
  'start': 13,
  'end': 18},
 {'entity': 'B-CLINICAL_EVENT',
  'score': 0.82905316,
  'index': 8,
  'word': '▁presented',
  'start': 18,
  'end': 28},
 {'entity': 'B-NONBIOLOGICAL_LOCATION',
  'score': 0.44632402,
  'index': 11,
  'word': '▁physician',
  'start': 35,
  'end': 45},
 {'entity': 'B-SIGN_SYMPTOM',
  'score': 0.9661406,
  'index': 15,
  'word': '▁fever',
  'start': 62,
  'end': 68},
 {'entity': 'B-SIGN

In [ ]:
result = {}
for item in ner_results:
    entity = item['entity']
    word = item['word'].strip('▁')  # Remove the leading '▁' for better readability
    result[entity] = word

# Verify the result
print(result)

{'B-AGE': '22', 'I-AGE': 'old', 'B-SEX': 'male', 'B-CLINICAL_EVENT': 'presented', 'B-NONBIOLOGICAL_LOCATION': 'physician', 'B-SIGN_SYMPTOM': 'symptoms', 'I-SIGN_SYMPTOM': 'throat', 'B-BIOLOGICAL_STRUCTURE': 'neck', 'I-BIOLOGICAL_STRUCTURE': 'nodes', 'I-DURATION': 'hours', 'B-SEVERITY': 'very'}


In [ ]:
from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
repo_id="mistralai/Mistral-7B-Instruct-v0.3"
llm=HuggingFaceEndpoint(repo_id=repo_id, max_length=200, temperature=0.5)

                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [ ]:
inputs = {
    "description": description,
    "result": result
}

In [ ]:
from langchain import LLMChain , PromptTemplate

template = """you are a healthcare provider. User enter their case history in  {description} variable
and model extract word and entities in {result} variable from that description. You're task isto understand description and
refine necessary word and entities. Refine by remove duplicates, combine same entities, add entities if find.
 """

In [ ]:
prompt= PromptTemplate(template=template, input_variables=["description", "result"])

In [ ]:
print(prompt)

input_variables=['description', 'result'] template="you are a healthcare provider. User enter their case history in  {description} variable\nand model extract word and entities in {result} variable from that description. You're task isto understand description and\nrefine necessary word and entities. Refine by remove duplicates, combine same entities, add entities if find.\n "


In [ ]:
llm_chain=LLMChain(llm=llm, prompt=prompt)

In [ ]:
refined_result=llm_chain.invoke(inputs)

In [ ]:
description = refined_result['description'],
text = refined_result['text']

# Display the extracted values
print("Description:")
print(description)
print("\nText:")
print(text)

Description:
A 22-year-old male presented to his physician complaining of a fever and sore throat with enlarged, tender lymph nodes in his neck for the last 24 hours. He was now feeling very ill, and his symptoms were getting worse.

Text:
 {'B-AGE': '22', 'I-AGE': 'old', 'B-SEX': 'male', 'B-CLINICAL_EVENT': 'presented', 'B-NONBIOLOGICAL_LOCATION': 'physician', 'B-SIGN_SYMPTOM': ['fever', 'sore throat'], 'B-BIOLOGICAL_LOCATION': ['neck'], 'B-LYMPH_NODE': ['nodes'], 'B-DURATION': '24 hours', 'B-SEVERITY': ['very ill', 'worsening symptoms']}


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [ ]:
%%writefile app.py

Overwriting app.py


In [ ]:
! wget -q -O - ipv4.icanhazip.com

35.236.139.241


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

⠙

⠹⠸⠼⠴⠦⠧⠇⠏Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.139.241:8501

